# Profiling compilation

## Modified compilation commands

In [ ]:
%%file tmp.fibo1.cpp

#include <iostream>

template<int N>
struct fibonacci {
  static int const value = fibonacci<N-1>::value + fibonacci<N-2>::value ;
} ;

template<>
struct fibonacci<1> {
  static int const value = 1 ;
} ;

template<>
struct fibonacci<0> {
  static int const value = 0 ;
} ;

int main() {
  std::cout << fibonacci<35>::value << std::endl ;
  return 0 ;
}

In [ ]:
!\time -f "%U s" g++ -O2 -std=c++20 tmp.fibo1.cpp -o tmp.fibo1.exe

In [ ]:
%%file tmp.fibo2.cpp

#include <iostream>

consteval int fibonacci( int n ) {
  if (n>1) return fibonacci(n-1) + fibonacci(n-2) ;
  else return n ;
}

int main() {
  std::cout << fibonacci(35) << std::endl ;
  return 0 ;
}

In [ ]:
!\time -f "%U s" g++ -O2 -std=c++20 -fconstexpr-ops-limit=333333333 tmp.fibo2.cpp -o tmp.fibo2.exe

In [ ]:
%%file tmp.fibo3.cpp

#include <iostream>
#include <vector>

consteval int fibonacci( int n ) {
  std::vector<int> values(n+1) ;
  values[0] = 0 ;
  values[1] = 1 ;
  for ( int i = 2 ; i <= n ; ++i ) {
    values[i] = values[i-1] + values[i-2] ;
  }
  return values[n] ;
}

int main() {
  std::cout << fibonacci(35) << std::endl ;
  return 0 ;
}

In [ ]:
!\time -f "%U s" g++ -O0 -std=c++20 tmp.fibo3.cpp -o tmp.fibo3.exe

## Measurements

Here are the results for `g++ 13.2 -std=c++20 -O2`, measured with GNU time on a processor i7-10875H, and with BuildBench.

|                     | Compilation time | BuildBench |
| :------------------ | -----: | --: |
| struct template     | 0.34 s | 0.6 |
| recursive function  | 1.86 s | 3.0 |
| bottom-up function  | 0.35 s | 0.7 |

© *CNRS 2024*
*Assembled and written in french by David Chamont, this work is made available according to the terms of the [Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*